In [6]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
MINUTO_ANALISADO = 10

In [7]:
df = pd.read_csv('../../../data/crawler/unified-events-time-statistics.csv')
filtro = df['time'] == MINUTO_ANALISADO
df = df[filtro]
df

,golId,time,result,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,blueJungleWR,blueJungleKDA,blueMidGP,...,RED:third_tower_top,RED:third_tower_mid,RED:third_tower_bot,RED:inhibitor_top,RED:inhibitor_mid,RED:inhibitor_bot,RED:baron,RED:elder_dragon,RED:nexus_tower,RED:nexus
10,35797,10,0,0,0.00,0.00,0,0.00,0.00,0,...,0,0,0,0,0,0,0,0,0,0
51,35798,10,0,0,0.00,0.00,2,0.00,1.18,0,...,0,0,0,0,0,0,0,0,0,0
100,35799,10,1,2,0.00,1.00,5,1.00,5.78,0,...,0,0,0,0,0,0,0,0,0,0
139,35800,10,1,3,0.33,4.25,0,0.00,0.00,0,...,0,0,0,0,0,0,0,0,0,0
174,35801,10,0,0,0.00,0.00,3,0.33,2.08,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195350,49142,10,0,2,0.50,3.67,8,0.62,5.18,0,...,0,0,0,0,0,0,0,0,0,0
195379,53243,10,0,4,0.25,2.83,10,0.80,8.38,2,...,0,0,0,0,0,0,0,0,0,0
195410,53244,10,1,5,0.80,2.91,3,1.00,3.75,10,...,0,0,0,0,0,0,0,0,0,0
195443,45053,10,0,0,0.00,0.00,2,0.00,4.71,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
y = df['result'].copy()
X = df.drop(['golId','result','time'],axis=1)
X.head()

,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,blueJungleWR,blueJungleKDA,blueMidGP,blueMidWR,blueMidKDA,blueADCGP,...,RED:third_tower_top,RED:third_tower_mid,RED:third_tower_bot,RED:inhibitor_top,RED:inhibitor_mid,RED:inhibitor_bot,RED:baron,RED:elder_dragon,RED:nexus_tower,RED:nexus
10,0,0.00,0.00,0,0.00,0.00,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
51,0,0.00,0.00,2,0.00,1.18,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
100,2,0.00,1.00,5,1.00,5.78,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
139,3,0.33,4.25,0,0.00,0.00,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
174,0,0.00,0.00,3,0.33,2.08,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,stratify=y,random_state=42)
    scaler = StandardScaler()   
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test,scaler

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

executions = pd.DataFrame()

for i in range(50):
  print('execução número ' + str(i))
  X_train,X_test,y_train,y_test,scaler = preprocess_input(X,y)

  models = {
      'Logistic Regression': LogisticRegression(max_iter=10000),
      'Support Vector Machine (RBF Kernel)': SVC(C=1,gamma=0.01,kernel='sigmoid',max_iter=50000,probability=True),
      'Decission Tree': DecisionTreeClassifier(),
      'Adaboost': AdaBoostClassifier(),
      'Random Forest': RandomForestClassifier(),
      'Gradient Boosting Classifier': GradientBoostingClassifier(),
      'KNN': KNeighborsClassifier(n_neighbors=3),
      'Gaussian NB': GaussianNB()
  }

  for name, model in models.items():
    model.fit(X_train,y_train)

  scores_list = []

  for name,model in models.items():    
      scores_list.append({
      'model': name,
      'game_minutes': MINUTO_ANALISADO,
      'execution': i,
      'Balanced Accuracy': balanced_accuracy_score(y_test,model.predict(X_test)),
      'Precision':  precision_score(y_test,model.predict(X_test)),
      'Recall': recall_score(y_test,model.predict(X_test)),
      'F1-Score': f1_score(y_test,model.predict(X_test)),
      'auc': roc_auc_score(y_test,model.predict_proba(X_test)[:,1])
      })
  # scores = pd.DataFrame(scores_list)
  executions = executions.append(scores_list)

execução número 0
execução número 1
execução número 2
execução número 3
execução número 4
execução número 5
execução número 6
execução número 7
execução número 8
execução número 9
execução número 10
execução número 11
execução número 12
execução número 13
execução número 14
execução número 15
execução número 16
execução número 17
execução número 18
execução número 19
execução número 20
execução número 21
execução número 22
execução número 23
execução número 24
execução número 25
execução número 26
execução número 27
execução número 28
execução número 29
execução número 30
execução número 31
execução número 32
execução número 33
execução número 34
execução número 35
execução número 36
execução número 37
execução número 38
execução número 39
execução número 40
execução número 41
execução número 42
execução número 43
execução número 44
execução número 45
execução número 46
execução número 47
execução número 48
execução número 49


In [11]:
avg_scores_list = []

for name, model in models.items():

    avg_balanced_acc = 0

    model_metrics = executions.loc[executions['model'] == name]
    avg_balanced_acc = model_metrics['Balanced Accuracy'].sum() / len(model_metrics['Balanced Accuracy'])
    avg_precision = model_metrics['Precision'].sum() / len(model_metrics['Precision'])
    avg_recall = model_metrics['Recall'].sum() / len(model_metrics['Recall'])
    avg_f_score = model_metrics['F1-Score'].sum() / len(model_metrics['F1-Score'])
    avg_auc = model_metrics['auc'].sum() / len(model_metrics['auc'])

    avg_scores_list.append({
      'model': name,
      'model_obj': model,
      'game_minutes': MINUTO_ANALISADO,
      'Balanced Accuracy': avg_balanced_acc,
      'Precision': avg_precision,
      'Recall': avg_recall,
      'F1-Score': avg_f_score,
      'auc': avg_auc
      })
avg_scores = pd.DataFrame(avg_scores_list)

In [12]:
ordered_scores = avg_scores.sort_values(by='auc', ascending=False)

In [13]:
ordered_scores

,model,model_obj,game_minutes,Balanced Accuracy,Precision,Recall,F1-Score,auc
5,Gradient Boosting Classifier,([DecisionTreeRegressor(criterion='friedman_ms...,10,0.664587,0.667386,0.719522,0.692474,0.719986
0,Logistic Regression,LogisticRegression(max_iter=10000),10,0.649206,0.652941,0.708134,0.679419,0.704825
4,Random Forest,"(DecisionTreeClassifier(max_features='auto', r...",10,0.651903,0.660277,0.690750,0.675141,0.704309
3,Adaboost,"(DecisionTreeClassifier(max_depth=1, random_st...",10,0.652254,0.654070,0.717703,0.684411,0.703644
1,Support Vector Machine (RBF Kernel),"SVC(C=1, gamma=0.01, kernel='sigmoid', max_ite...",10,0.644936,0.648609,0.706539,0.676336,0.700358
7,Gaussian NB,GaussianNB(),10,0.509893,0.640000,0.051037,0.094535,0.691604
6,KNN,KNeighborsClassifier(n_neighbors=3),10,0.585734,0.601881,0.612440,0.607115,0.610921
2,Decission Tree,DecisionTreeClassifier(),10,0.582479,0.600348,0.599298,0.599800,0.581142


In [14]:
# Ensemble Voting Classifier
from sklearn.ensemble import VotingClassifier

# Criando o ensemble com os classificadores
ensemble_classifier = VotingClassifier(estimators=[
    (ordered_scores.iloc[0]['model'], ordered_scores.iloc[0]['model_obj']),
    (ordered_scores.iloc[1]['model'], ordered_scores.iloc[1]['model_obj']),
    (ordered_scores.iloc[2]['model'], ordered_scores.iloc[2]['model_obj'])
], voting='soft')

ensemble_classifier = ensemble_classifier.fit(X_train, y_train)

In [15]:
ensemble_scores_list = []
ensemble_scores_list.append({
    'model': 'Ensemble Classifier',
    'model_obj': ensemble_classifier,
    'game_minutes': MINUTO_ANALISADO,
    'Balanced Accuracy': balanced_accuracy_score(y_test,ensemble_classifier.predict(X_test)),
    'Precision':  precision_score(y_test,ensemble_classifier.predict(X_test)),
    'Recall': recall_score(y_test,ensemble_classifier.predict(X_test)),
    'F1-Score': f1_score(y_test,ensemble_classifier.predict(X_test)),
    'auc': roc_auc_score(y_test,ensemble_classifier.predict_proba(X_test)[:,1])
})

In [16]:
ensemble_scores = pd.DataFrame(ensemble_scores_list)
ensemble_scores_list

[{'model': 'Ensemble Classifier',
  'model_obj': VotingClassifier(estimators=[('Gradient Boosting Classifier',
                                GradientBoostingClassifier()),
                               ('Logistic Regression',
                                LogisticRegression(max_iter=10000)),
                               ('Random Forest', RandomForestClassifier())],
                   voting='soft'),
  'game_minutes': 10,
  'Balanced Accuracy': 0.6633273524720893,
  'Precision': 0.6651982378854625,
  'Recall': 0.722488038277512,
  'F1-Score': 0.6926605504587156,
  'auc': 0.718124224703172}]

In [17]:
ensemble_scores

,model,model_obj,game_minutes,Balanced Accuracy,Precision,Recall,F1-Score,auc
0,Ensemble Classifier,VotingClassifier(estimators=[('Gradient Boosti...,10,0.663327,0.665198,0.722488,0.692661,0.718124


In [18]:
models = pd.concat([ensemble_scores,avg_scores])
ordered_models = models.sort_values(by='auc', ascending=False)
best_model = ordered_models.iloc[0]['model_obj']
best_model

GradientBoostingClassifier()

In [19]:
header = ['model', 'game_minutes', 'auc']
models.to_csv('../../../data/models-analysis/models-time-metric.csv', columns = header, mode='a',index=False,header=False)